In [1]:
# code taken from here: https://medium.com/swlh/tweet-sentiment-analysis-using-python-for-complete-beginners-4aeb4456040
# code also influenced from here: https://www.kaggle.com/code/ragnisah/text-data-cleaning-tweets-analysis/notebook
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
Tweet_Data = pd.read_csv('Tweet_Data.csv')

stop_words=stopwords.words('english')
stemmer=PorterStemmer()

In [2]:
Tweet_Data.shape 

(50854, 5)

In [3]:
def remove_usernames_links(tweet):
    tweet = re.sub('@[^\s]+','',tweet)
    tweet = re.sub('http[^\s]+','',tweet)
    return tweet
Tweet_Data['Tweet'] = Tweet_Data['Tweet'].apply(remove_usernames_links)
Tweet_Data.shape

(50854, 5)

In [4]:
# code from here: https://s-hosseinkhani1999.medium.com/remove-all-kind-of-emojis-with-the-demoji-package-python-643a530491f4
import demoji
demoji.download_codes()
def remove_em(text):
    dem = demoji.findall(text)
    for item in dem.keys():
        text = text.replace(item, '')
    return text
Tweet_Data['Tweet'] = Tweet_Data['Tweet'].apply(remove_em)
Tweet_Data.shape

/var/folders/j1/xwlyd2kd5hb4q6nbz5k6s9v80000gn/T/ipykernel_92533/2987164781.py:3: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


(50854, 5)

In [5]:
# code taken from here:https://www.analyticsvidhya.com/blog/2021/06/text-preprocessing-in-nlp-with-python-codes/

#library that contains punctuation
import string
string.punctuation

#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
Tweet_Data['clean_tweet']= Tweet_Data['Tweet'].apply(lambda x:remove_punctuation(x))
Tweet_Data.shape

(50854, 6)

In [6]:
Tweet_Data['tweet_lower']= Tweet_Data['clean_tweet'].apply(lambda x: x.lower())
Tweet_Data.head()

,Unnamed: 0,Date,User,Tweet,View,clean_tweet,tweet_lower
0,0,2019-12-31 18:10:55+00:00,OwenJones84,yes yes it is,Liberal,yes yes it is,yes yes it is
1,1,2019-12-31 18:05:32+00:00,OwenJones84,"It has been an absolute shitter of a year, in multiple ways, and there's no getting around that....",Liberal,It has been an absolute shitter of a year in multiple ways and theres no getting around that\n\n...,it has been an absolute shitter of a year in multiple ways and theres no getting around that\n\n...
2,2,2019-12-31 17:54:53+00:00,OwenJones84,"hairdresser's away until 6th January, nightmare",Liberal,hairdressers away until 6th January nightmare,hairdressers away until 6th january nightmare
3,3,2019-12-31 17:51:20+00:00,OwenJones84,"#2009vs2019 \n\nTen years on and here I am, still looking like a budget Macaulay Culkin",Liberal,2009vs2019 \n\nTen years on and here I am still looking like a budget Macaulay Culkin,2009vs2019 \n\nten years on and here i am still looking like a budget macaulay culkin
4,4,2019-12-31 16:41:34+00:00,OwenJones84,The most predictable thing in the world is anti-trans activists rushing to defend homophobic cra...,Liberal,The most predictable thing in the world is antitrans activists rushing to defend homophobic crap...,the most predictable thing in the world is antitrans activists rushing to defend homophobic crap...


In [7]:
#defining function for tokenization
import re
from nltk.tokenize import word_tokenize 
def tokenization(text):
    tokens = word_tokenize(text)
    return tokens
#applying function to the column
Tweet_Data['tweet_tokenised']= Tweet_Data['tweet_lower'].apply(lambda x: tokenization(x))
Tweet_Data.shape

(50854, 8)

In [8]:
#importing nlp library
import nltk
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

Tweet_Data['no_stopwords']= Tweet_Data['tweet_tokenised'].apply(lambda x:remove_stopwords(x))


In [9]:
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()

def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text
Tweet_Data['tweet_stemmed']=Tweet_Data['no_stopwords'].apply(lambda x: stemming(x))
Tweet_Data.shape

(50854, 10)

In [10]:
#nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
Tweet_Data['tweet_lemmatized']=Tweet_Data['no_stopwords'].apply(lambda x:lemmatizer(x))
Tweet_Data.shape

(50854, 11)

In [11]:
import gensim 
from gensim.models import Word2Vec
from gensim.models import KeyedVectors


training_docs = Tweet_Data['tweet_lemmatized']
model= Word2Vec(training_docs,size=300,workers=4)

In [12]:
# function taken from here: https://github.com/PradipNichite/Youtube-Tutorials/blob/main/Yotutube_WordVectors.ipynb
def sent_vec_corpus(sent):
    vector_size = model.wv.vector_size
    model.wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in model.wv:
            ctr += 1
            model.wv_res += model.wv[w]
    model.wv_res = model.wv_res/ctr
    return model.wv_res

In [14]:
Tweet_Data['trained_vec'] = Tweet_Data['tweet_lemmatized'].apply(sent_vec_corpus)
print(Tweet_Data['trained_vec'])

0        [0.20818471908569336, -0.3044995864232381, 0.017170158525307972, 0.18408364057540894, -0.3491394...
1        [0.12491666475155701, -0.1480059965979308, -0.01796685206257583, 0.12862281793301614, -0.1037743...
2        [0.03663999186828733, -0.04670585785061121, 0.002301126392558217, 0.031321539729833606, -0.03373...
3        [0.1574381380341947, -0.15598000373159135, 0.055554031394422054, 0.06790702718509627, -0.0557207...
4        [0.034517355069207646, -0.11943318322300911, 0.05595608373793463, 0.023727972849479154, -0.08448...
                                                        ...                                                 
50849    [-0.023258453840389846, -0.08174765892326832, -0.10858170306310058, 0.12391643971204758, -0.0295...
50850    [0.1837572589283809, -0.2890149224549532, 0.08148819714551792, 0.16420193493831903, -0.298969227...
50851    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
50852    [0.2600117

In [15]:
views = pd.get_dummies(Tweet_Data['View'])
Tweet_Data2 = Tweet_Data.join(views)
Tweet_Data2

,Unnamed: 0,Date,User,Tweet,View,clean_tweet,tweet_lower,tweet_tokenised,no_stopwords,tweet_stemmed,tweet_lemmatized,trained_vec,Conservative,Liberal
0,0,2019-12-31 18:10:55+00:00,OwenJones84,yes yes it is,Liberal,yes yes it is,yes yes it is,"[yes, yes, it, is]","[yes, yes]","[ye, ye]","[yes, yes]","[0.20818471908569336, -0.3044995864232381, 0.017170158525307972, 0.18408364057540894, -0.3491394...",0,1
1,1,2019-12-31 18:05:32+00:00,OwenJones84,"It has been an absolute shitter of a year, in multiple ways, and there's no getting around that....",Liberal,It has been an absolute shitter of a year in multiple ways and theres no getting around that\n\n...,it has been an absolute shitter of a year in multiple ways and theres no getting around that\n\n...,"[it, has, been, an, absolute, shitter, of, a, year, in, multiple, ways, and, theres, no, getting...","[absolute, shitter, year, multiple, ways, theres, getting, around, fought, world, thats, decent,...","[absolut, shitter, year, multipl, way, there, get, around, fought, world, that, decent, happi, h...","[absolute, shitter, year, multiple, way, there, getting, around, fought, world, thats, decent, h...","[0.12491666475155701, -0.1480059965979308, -0.01796685206257583, 0.12862281793301614, -0.1037743...",0,1
2,2,2019-12-31 17:54:53+00:00,OwenJones84,"hairdresser's away until 6th January, nightmare",Liberal,hairdressers away until 6th January nightmare,hairdressers away until 6th january nightmare,"[hairdressers, away, until, 6th, january, nightmare]","[hairdressers, away, 6th, january, nightmare]","[hairdress, away, 6th, januari, nightmar]","[hairdresser, away, 6th, january, nightmare]","[0.03663999186828733, -0.04670585785061121, 0.002301126392558217, 0.031321539729833606, -0.03373...",0,1
3,3,2019-12-31 17:51:20+00:00,OwenJones84,"#2009vs2019 \n\nTen years on and here I am, still looking like a budget Macaulay Culkin",Liberal,2009vs2019 \n\nTen years on and here I am still looking like a budget Macaulay Culkin,2009vs2019 \n\nten years on and here i am still looking like a budget macaulay culkin,"[2009vs2019, ten, years, on, and, here, i, am, still, looking, like, a, budget, macaulay, culkin]","[2009vs2019, ten, years, still, looking, like, budget, macaulay, culkin]","[2009vs2019, ten, year, still, look, like, budget, macaulay, culkin]","[2009vs2019, ten, year, still, looking, like, budget, macaulay, culkin]","[0.1574381380341947, -0.15598000373159135, 0.055554031394422054, 0.06790702718509627, -0.0557207...",0,1
4,4,2019-12-31 16:41:34+00:00,OwenJones84,The most predictable thing in the world is anti-trans activists rushing to defend homophobic cra...,Liberal,The most predictable thing in the world is antitrans activists rushing to defend homophobic crap...,the most predictable thing in the world is antitrans activists rushing to defend homophobic crap...,"[the, most, predictable, thing, in, the, world, is, antitrans, activists, rushing, to, defend, h...","[predictable, thing, world, antitrans, activists, rushing, defend, homophobic, crap, press, thro...","[predict, thing, world, antitran, activist, rush, defend, homophob, crap, press, throw, grotesqu...","[predictable, thing, world, antitrans, activist, rushing, defend, homophobic, crap, press, throw...","[0.034517355069207646, -0.11943318322300911, 0.05595608373793463, 0.023727972849479154, -0.08448...",0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50849,274,2019-01-04 09:55:03+00:00,MelanieLatest,The pathological animus of the New York Times,Conservative,The pathological animus of the New York Times,the pathological animus of the new york times,"[the, pathological, animus, of, the, new, york, times]","[pathological, animus, new, york, times]","[patholog, animu, new, york, time]","[pathological, animus, new, york, time]","[-0.023258453840389846, -0.08174765892326832, -0.10858170306310058, 0.12391643971204758, -0.0295...",1,0
50850,275,2019-01-02 09:49:31+00:00,MelanieLatest,"Eh? Have never even written about any

In [16]:
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate

svm = SVC()
X = Tweet_Data2['trained_vec'].tolist()
y = Tweet_Data2['Conservative']

cv_results = cross_validate(svm, X, y, cv=5, scoring=('accuracy','precision','recall','f1'))
print(cv_results)



{'fit_time': array([ 499.15358806, 1492.34156632, 2219.74478006,  580.86125326,
       1090.08681703]), 'score_time': array([170.34050775, 178.61075568, 211.04377103, 182.54281282,
       223.75830507]), 'test_accuracy': array([0.56621768, 0.62855176, 0.66276669, 0.57093698, 0.53588987]), 'test_precision': array([0.46912078, 0.60150376, 0.60072314, 0.47958643, 0.45448067]), 'test_recall': array([0.3663578 , 0.304038  , 0.55249406, 0.42969121, 0.6060822 ]), 'test_f1': array([0.41141942, 0.40391291, 0.5756001 , 0.45326986, 0.51944614])}


In [17]:
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression(random_state=0)

cv_logistic_results = cross_validate(logistic_regression, X, y, cv=5,scoring=('accuracy','precision','recall','f1'))
print(cv_logistic_results)


/Users/douglaswood/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/douglaswood/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

{'fit_time': array([3.7403779 , 2.87768292, 4.28687501, 3.4126451 , 3.0539701 ]), 'score_time': array([0.08297491, 0.04229498, 0.05585408, 0.05405092, 0.06726193]), 'test_accuracy': array([0.51882804, 0.57880248, 0.61006784, 0.5694622 , 0.53048181]), 'test_precision': array([0.38062043, 0.48261278, 0.54335466, 0.47180514, 0.43494253]), 'test_recall': array([0.25944405, 0.24394299, 0.3631829 , 0.33586698, 0.44951295]), 'test_f1': array([0.30856174, 0.324077  , 0.43536446, 0.39239628, 0.44210772])}


/Users/douglaswood/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
from sklearn.naive_bayes import ComplementNB
from sklearn.preprocessing import MinMaxScaler

NB = ComplementNB()
scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)


cv_NB_results = cross_validate(NB, X_scaled, y, cv=5,scoring=('accuracy','precision','recall','f1'))
print(cv_NB_results)

{'fit_time': array([0.11916804, 0.10040498, 0.09085894, 0.15206504, 0.09246397]), 'score_time': array([0.03842092, 0.03314376, 0.04152298, 0.04218864, 0.02896285]), 'test_accuracy': array([0.5081113 , 0.65971881, 0.64959198, 0.58568479, 0.47522124]), 'test_precision': array([0.40805929, 0.57059378, 0.55672638, 0.49965446, 0.42498005]), 'test_recall': array([0.41862675, 0.71900238, 0.75296912, 0.68693587, 0.75908767]), 'test_f1': array([0.41327548, 0.63625854, 0.6401454 , 0.5785157 , 0.54489639])}
